In [1]:
import numpy as np
import tensorflow as tf
from keras import layers
from PIL import Image
from IPython.display import display
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.models import Model
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt


In [2]:
target_image_size = (75, 75)
model_name = ''
num_classes = 17


In [ ]:
# # 讀取資料集
# x_train = np.load('train_image.npy')
# y_train = np.load('train_label.npy')
# x_test = np.load('test_image.npy')
# y_test = np.load('test_label.npy')

In [3]:

train_datagen = ImageDataGenerator(
    rescale=1./255,  # 对图像进行缩放，将像素值归一化到[0,1]之间
    rotation_range=30,  # 随机旋转角度范围
    width_shift_range=0.2,  # 随机水平平移的范围
    height_shift_range=0.2,  # 随机竖直平移的范围
    shear_range=0.1,  # 剪切强度
    zoom_range=0.2,  # 随机缩放范围
    horizontal_flip=False,  # 随机水平翻转
    fill_mode='nearest'  # 填充方式
)

train_dir = 'Dataset/train/'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_image_size,
    batch_size=32,
    class_mode='categorical'
)

# 创建验证集的 ImageDataGenerator 对象
validation_datagen =  ImageDataGenerator(rescale=1./255)

val_dir = 'Dataset/val/'

# 加载验证集
validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=target_image_size,
    batch_size=32,
    class_mode='categorical'
)

Found 1485 images belonging to 17 classes.
Found 476 images belonging to 17 classes.


In [ ]:
# # 創建模型
# model = tf.keras.Sequential([
#     layers.Conv2D(64, (5, 5), activation='relu', input_shape=(128, 128, 3)),
#     layers.BatchNormalization(),
#     layers.MaxPooling2D((2, 2)),
#     layers.Dropout(0.3),
#     layers.Conv2D(128, (5, 5), activation='relu'),
#     layers.BatchNormalization(),
#     layers.MaxPooling2D((2, 2)),
#     layers.Dropout(0.3),
#     layers.Conv2D(256, (5, 5), activation='relu'),
#     layers.BatchNormalization(),
#     layers.MaxPooling2D((2, 2)),
#     layers.Dropout(0.3),
#     layers.Flatten(),
#     layers.Dense(128, activation='relu'),
#     layers.BatchNormalization(),
#     layers.Dropout(0.5),
#     layers.Dense(17, activation='softmax')
# ])


# checkpoint = ModelCheckpoint(filepath='model.h5', monitor='val_accuracy', save_best_only=True)

# # 編譯模型
# model.compile(
#     optimizer='adam',
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

In [ ]:
if os.path.exists('model.h5'):
    os.remove('model.h5')

In [4]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

model_name = 'InceptionResNetV2'
# 建立 InceptionResNetV2 模型，載入 ImageNet 預訓練權重
input_shape = (target_image_size[0], target_image_size[1], 3)
num_classes = 17
base_model = InceptionResNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

# 添加自定義的輸出層
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

# 構建完整模型
model = Model(inputs=base_model.input, outputs=predictions)

# 鎖定 InceptionResNetV2 的卷積層權重，只訓練自定義的輸出層
for layer in base_model.layers:
    layer.trainable = False

checkpoint = ModelCheckpoint(filepath='model.h5', monitor='val_accuracy', save_best_only=True)   
    
# 編譯模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2023-05-17 10:07:00.049066: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-17 10:07:00.049131: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-17 10:07:00.049162: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (t741051-virtual-machine): /proc/driver/nvidia/version does not exist
2023-05-17 10:07:00.049444: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
from keras.applications.mobilenet import MobileNet
from keras.models import Model
model_name = 'MobileNet'
# 建立 MobileNet 模型，載入 ImageNet 預訓練權重
input_shape = (target_image_size[0], target_image_size[1], 3)
base_model = MobileNet(input_shape=input_shape, include_top=False, weights='imagenet')

# 添加自定義的輸出層
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

# 構建完整模型
model = Model(inputs=base_model.input, outputs=predictions)

# 鎖定 MobileNet 的卷積層權重，只訓練自定義的輸出層
for layer in base_model.layers:
    layer.trainable = False

checkpoint = ModelCheckpoint(filepath='model.h5', monitor='val_accuracy', save_best_only=True)   
    
# 編譯模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# 載入訓練和測試數據
train_data = np.load('train_image.npy')
train_labels = np.load('train_label.npy')
test_data = np.load('test_image.npy')
test_labels = np.load('test_label.npy')


train_data = train_data.reshape(train_data.shape[0], -1)
test_data = test_data.reshape(test_data.shape[0], -1)

# Create the decision tree model
model = DecisionTreeClassifier()

# Train the model on the training data
model.fit(train_data, train_labels)

# Evaluate the model on the testing data
score = model.score(test_data, test_labels)

# Print the accuracy score
print('Accuracy:', score)

In [ ]:
# 訓練模型
history = model.fit(train_generator, epochs=30, validation_data=validation_generator, callbacks=checkpoint)

In [ ]:
print(model_name)
val_acc = max(history.history['val_accuracy'])
train_acc = history.history['accuracy'][history.history['val_accuracy'].index(val_acc)]
print("val_max：", val_acc)
print("train：", train_acc)